## Read data from json files and save to a CSV

In [1]:
import json
import csv
from os import close, read
import pandas as pd
from csv import writer

 
OMP_PROC_BIND = ['close', 'spread']
OMP_PLACES = ['cores', 'sockets', 'threads']

counters = [       "skx_unc_imc0::UNC_M_CAS_COUNT:RD:cpu=0",
                   "skx_unc_imc1::UNC_M_CAS_COUNT:RD:cpu=0",
                   "skx_unc_imc2::UNC_M_CAS_COUNT:RD:cpu=0",
                   "skx_unc_imc3::UNC_M_CAS_COUNT:RD:cpu=0",
                   "skx_unc_imc4::UNC_M_CAS_COUNT:RD:cpu=0",
                   "skx_unc_imc5::UNC_M_CAS_COUNT:RD:cpu=0",
                   "skx_unc_imc0::UNC_M_CAS_COUNT:RD:cpu=23",
                   "skx_unc_imc1::UNC_M_CAS_COUNT:RD:cpu=23",
                   "skx_unc_imc2::UNC_M_CAS_COUNT:RD:cpu=23",
                   "skx_unc_imc3::UNC_M_CAS_COUNT:RD:cpu=23",
                   "skx_unc_imc4::UNC_M_CAS_COUNT:RD:cpu=23",
                   "skx_unc_imc5::UNC_M_CAS_COUNT:RD:cpu=23",
                   "skx_unc_imc0::UNC_M_CAS_COUNT:WR:cpu=0",
                   "skx_unc_imc1::UNC_M_CAS_COUNT:WR:cpu=0",
                   "skx_unc_imc2::UNC_M_CAS_COUNT:WR:cpu=0",
                   "skx_unc_imc3::UNC_M_CAS_COUNT:WR:cpu=0",
                   "skx_unc_imc4::UNC_M_CAS_COUNT:WR:cpu=0",
                   "skx_unc_imc5::UNC_M_CAS_COUNT:WR:cpu=0", 
                   "skx_unc_imc0::UNC_M_CAS_COUNT:WR:cpu=23",
                   "skx_unc_imc1::UNC_M_CAS_COUNT:WR:cpu=23",
                   "skx_unc_imc2::UNC_M_CAS_COUNT:WR:cpu=23",
                   "skx_unc_imc3::UNC_M_CAS_COUNT:WR:cpu=23",
                   "skx_unc_imc4::UNC_M_CAS_COUNT:WR:cpu=23",
                   "skx_unc_imc5::UNC_M_CAS_COUNT:WR:cpu=23"
]

column_name = ['Read', 'Write']
with open('results.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(column_name)

for bind in OMP_PROC_BIND:
    for places in OMP_PLACES:
        snapshot1 = []
        snapshot2 = []
        snapshot3 = []
        snapshot4 = []
        snapshot5 = []
        index = 0
        for counter in counters:
            f = open(bind + " " + places + '/cali_p2z_omp_t10/cali_' + counter + '.json')
            data = json.load(f)
            index = 0
            for i in data:
                if 'event.end#annotation' in i:
                    if index == 0:
                        snapshot1.append(i['papi.' + counter])
                    elif index == 1:
                        snapshot2.append(i['papi.' + counter])
                    elif index == 2:
                        snapshot3.append(i['papi.' + counter])
                    elif index == 3:
                        snapshot4.append(i['papi.' + counter])
                    elif index == 4:
                        snapshot5.append(i['papi.' + counter])
                    index+=1

        def Average(lst):
            return sum(lst) / len(lst)

        def sum_of_list(l):
            total = 0
            for val in l:
                total = total + val
            return total

        sum = []


        with open('data.csv', 'a') as f:
                writer = csv.writer(f)
                writer.writerow(counters)
                writer.writerow(snapshot1)
                writer.writerow(snapshot2)
                writer.writerow(snapshot3)
                writer.writerow(snapshot4)
                writer.writerow(snapshot5)

                
        df = pd.read_csv("data.csv", usecols = counters)
        for counter in counters:
            sum.append(df[counter].sum() * 64)

        lst = []
        lst1  = sum[0:12]
        lst2  = sum[12:24]
        lst.append(sum_of_list(lst1))
        lst.append(sum_of_list(lst2))

        tmp = []
        with open('results.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(lst)

        import os
        os.remove("data.csv")

FileNotFoundError: ignored

## Create Bar Charts

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
  

df = pd.read_csv("results.csv", names=column_name)
print(df)

read_bytes = df.Read.to_list()
write_bytes = df.Write.to_list()
read_bytes.remove('Read')
write_bytes.remove('Write')

print(read_bytes)
print(write_bytes)

X = ['Close Cores','Close Sockets','Close Threads','Spread Cores','Spread Sockets','Spread Threads']
X_axis = np.arange(len(X))
  
plt.bar(X_axis - 0.2, write_bytes, 0.4, label = 'Write')
plt.bar(X_axis + 0.2, read_bytes, 0.4, label = 'Read')
  
plt.xticks(X_axis, X)
plt.xlabel("Groups")
plt.ylabel("Number of Bytes")
plt.title("OpenMP ICC 10 Threads")
plt.legend()
plt.show()

import os
os.remove("results.csv")